# Demystifying Neural Network
## Pytorch Tutorial 2

- Learn about `torch.nn`, `torch.optim`, `Dataset`, `DataLoader` features of Pytorch 

### Intro
- In the last Pytorch tutorial (part 3 of the series), I wrote about basic tensor operations. In this tutorial, I constructed a neural network to understand its architecture with MNIST digit dataset. I used this [excellent source](https://pytorch.org/tutorials/beginner/nn_tutorial.html) from Pytorch. This tutorial helped me the best in understanding neural network architecture as it first starts with only using basic tensor operations, but then gradually adding features such as  `torch.nn`, `torch.optim`, `Dataset`, `DataLoader`, so that one can see what each feature does. Since the source has excellent explanation, I tried to explain in Korean so that I am not merely just copying what is written in the tutorial. 


- Pytorch tutorial 1 에서는 간단한 tensor operation만 공부해 보았다. 이번 tutorial에서는 neural network 구조에 대해 공부하기 위해 직접 pytorch로 MNIST 숫자 데이터를 가지고 neural network를 만드는 연습을 해 보았다. Pytorch에서 제공하는 [tutorial](https://pytorch.org/tutorials/beginner/nn_tutorial.html)을 보고 실습을 해 보았는데, pytorch 기능을 하나도 사용 하지 않고 시작하여, 하나씩 하나씩 pytorch기능들을 추가로 넣어줌으로서 pytorch feature들을 하나씩 알아볼 수 있는 아주 좋은 tutorial이니 꼭 확인 해 보시길!

In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
from torch import nn, optim
from torch.autograd import Variable

In [2]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [3]:
import pickle
import gzip

with gzip.open((PATH/FILENAME).as_posix(), "rb") as f:
     ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

- convert our loaded data to tensor object using `map` function
- `map` 함수를 통해 로드 된 데이터를 텐서 object로 변환해 줍니다

In [4]:
import torch
x_train, y_train, x_valid, y_valid = map(torch.tensor, (x_train, y_train, x_valid, y_valid))

- 50000개의 hand written digit 이 있고, 28 x 28이미지는 flatten된 1d 텐서로 저장이 되어 있습니다 (1 row에 784)
- there are 50000 hand written digit images and each image is 28 x 28. Images are stored as a flattened 1d tensor (784)

In [15]:
# target value has 10 classes 0 to 9
x_train.shape, y_train.min(), y_train.max()

(torch.Size([50000, 784]), tensor(0), tensor(9))

## Neural Net 만들기 Create a model only using Pytorch tensor operation
- 기본적 pytorch tensor operation만을 사용하여 neural network를 만들어 보자. Weights와 bias를 만들기 위해 pytorch의 `torch.randn`을 사용하여 weights와 bias를 Xavier initialization으로 만들어 준다
- Let's make a neural network only using basic tensor operations. First, we make weights and bias tensors by using `torch.randn` function. I will initialize weights with Xavier initalization.
- Then, I'm going to use log softmax function to get prediction value, then use negative log likelihood as a loss function
- log softmax 함수를 사용하여 예측값을 구하고, loss function으로는 softmax의 짝꿍인 negative log likelihood를 사용한다 

In [22]:
import math
weights = torch.randn(784, 10) / math.sqrt(784) # Xavier initialization multiplies 1/sqrt(n)

- **after** initializing weights, we set `requires_grad_`. This function lets Pytorch to calculate the gradient during back propagation automatically.
- `requires_grad_`을 weights 텐서 object를 만든 후, 아래와 같이 실행을 시켜 준다 `requires_grad_`는 자동적으로 Pytorch가 back propagation을 할 때, gradient를 계산하도록 하는 스위치를 켜주는 역할을 한다
- `_` in Python signifies that the operation is performed in-place. There is also a different method to turn on `requires_grad` switch which is to give `requires_grad` parameter when creating tensor object. 
- Pytorch에서는 함수 뒤에 `_`가 붙으면 `inplace`역할을 합니다. bias에 대한 requires_grad를 실행해 줍니다. `requires_grad` 스위치를 켜줄 때 bias에서 만든 tensor처럼 스위치를 생성할때 안에 parameter로 줄 수 있다

In [23]:
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

- I haven't seen `@` before I read this tutorial, but it stands for the dot product operation.
- `@`는 pytorch tutorial을 읽으면서 처음 보았는데 벡터의 내적을 뜻한다

In [51]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1) 
# sum(-1) represents row sum and unsqueeze(-1) adds an additional dimension to the last index

def model(xb):
    return log_softmax(xb @ weights + bias) # @ represents dot product operation

- set batch size to 64. This represents one mini batch of 64 images. We call our function `model` on one batch of data. We just did one forward pass to get our predictions by passing in one batch of data. 
- 배치 사이즈를 64로 설정을 한다. 배치사이즈는 2의 거듭제곱으로 나아간다. GPU 성능에 따라 배치사이즈를 늘려갈 수 있다. 배치사이즈를 설정하고 위의 `model` function에 넣어 prediction을 구하면, 1 forward pass를 마친 것이다 

In [52]:
bs = 64 # batch size is normally power of 2 (64, 128, etc)
xb = x_train[0:bs] # one mini batch of 64 images
preds = model(xb)
print(preds[0], preds.shape)

tensor([-2.5335, -1.8878, -2.5305, -2.1693, -1.8341, -2.2710, -2.9698, -2.4828,
        -2.3980, -2.4458], grad_fn=<SelectBackward>) torch.Size([64, 10])


#### negative log likelihood function

- 아래 nll의 return 값을 보면 `-input[range(target.shape[0]), target].mean()`라고 되어있는데 여기서 `input`은 prediction값, `range(target.shape[0])`는 batch size인 64, `target`은 실제 target값이다. `input` 값이 log softmax로 prediction된 값 이므로 batch 64개의 `target` tensor object로 지정하여 index를 지정해 주면, 그 지정된 값으로 평균을 계산하여 negative log likelihood 값을 준다

In [55]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean() #target이 yb이고 각 index에 있는 것 mean
loss_func = nll # set log likelihood as a loss function
yb = y_train[0:bs]

In [65]:
print(loss_func(preds, yb))

tensor(2.3172, grad_fn=<NegBackward>)


In [193]:
print(loss_func(preds, yb).dtype, preds.shape, yb.shape)

torch.float32 torch.Size([64, 10]) torch.Size([64])


#### accuracy
- for each prediction, if the index with the largest value matches the target value, then the prediction is correct. to caculate accuracy of our neural network, we get that 
- accuracy 계산은 `torch.argmax`를 통해 prediction값 중, 가장 큰 값의 index가 target값과 일치할 때의 값을 평균낸다

In [81]:
a = torch.Tensor([1,2,3,4,5,6,1,2])
torch.argmax(a) # gives you the index

tensor(5)

In [85]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds==yb).float().mean()

In [86]:
print(accuracy(preds, yb)) # 처음에 설정한 weight 값이 random이므로 첫 batch를 돌린 후 accuracy가 낮음을 확인할 수 있다 

tensor(0.0625)


### Training loop. For each iteration :
### Training loop 돌기 : 
- epoch별, batch별로 loop을 돌려야 하므로 각 이터레이션에서 해야하는 일은:
    - batch size로 feed forward해줄 구간 설정
    - feed forward로 예측값 내기
    - loss 계산하기
    - `loss.backward()`로 그레디언트를 계산하기 (`weight`과 `bias`의 gradient가 된다)
    
    - select mini-batch of data
    - use the model to make predictions (feed forward)
    - calculate the loss
    - `loss.backward()` updates the gradients of the model, in this case, `weights` and `bias`
    
**`torch.no_grad()`** 안에서 weights과 bias의 gradient들을 update해야 한다. 그 이유는 아래의 부분이 gradient를 계산하는데 있어서 영향을 주면 안되기 때문이다. weights와 bias를 업데이트 하기 위함이지 gradient계산은 `loss.backward()`에서 끝났다. 
```
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
```
그 후, 마지막 단계에서 gradient들이 쌓여 다음 배치에 영향을 주는 것을 방지 하기위해 **`grad.zero_()`** 를 사용하여 gradient를 0으로 재 설정 시켜준다. 

In [88]:
n, c = x_train.shape

In [89]:
from IPython.core.debugger import set_trace #set_trace is useful for debugging in python
lr = 0.5
epochs = 2
for epoch in range(epochs):
    for i in range((n-1)//bs+1):
#         set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i: end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        with torch.no_grad():
            # because we do not want below actions to be recorded for our next calculation of the gradient
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

- without any hidden layer, we trained our first neural network. Let's compare the loss and accuracy after training and then those after only one batch of training. 
- hidden layer없이 첫 neural network를 train해 보았다. 전에 구했던 loss와 accuracy를 train이 끝난 후의 loss와 accuracy와 비교해 보자

In [90]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0812, grad_fn=<NegBackward>) tensor(1.)


### torch.nn.functional 사용하여 neural network 만들기
### using torch.nn.functional to build a neural network
- 위에서는 basic tensor operation으로 neural network를 만들어 보았지만, 이번에는 위에서 직접 써주었던 softmax function과 negative log likelihood function을 `torch.nn.functional` 에 있는 함수들로 바꾸어 본다. 대부분 `torch.nn.functional` 를 `F`로 쓴다

- 위에서 사용한 log softmax activation 과 negative log likelihood는 pytorch에서는 `F.cross_entropy` 함수를 제공해 주어 두개의 함수를 하나의 function으로 사용이 가능하다. 그러기에 위의 log softmax 함수를 쓰지 않아도 된다. 

In [91]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [92]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0812, grad_fn=<NllLossBackward>) tensor(1.)


### nn.Module과 nn.Parameter를 사용하기
### Refactor using nn.Module and nn.Parameter
- `nn.Module`과 `nn.Parameter`를 사용하면 training loop을 좀 더 깔끔하고 짧게 만들 수 있다. (여기서의 Module은 Python의 Module이 아님을 유의하자. `nn.Module`은 class이다)
- `nn.Module`을 상속받아 우리에게 필요한 weights, bias, 그리고 feed forward function을 가지고 있는 class를 만드는 것이 neural network function을 관리하는데 있어서 가장 효율적인 방법이다

In [199]:
from torch import nn

class Mnist_Logistic(nn.Module): # layer가 한개 밖에 없으므로 logistic regression과 같다
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10)/math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))
    
    def forward(self, xb):
        return xb @ self.weights + self.bias

In [200]:
model = Mnist_Logistic() # neural network object를 생성해 준다 

In [201]:
print(loss_func(model(xb), yb))

tensor(2.2093, grad_fn=<NllLossBackward>)


- 전 모델에서는 `weights`와 `bias`를 하나하나 업데이트 시켜 주었던 것과는 다르게, neural network를 object화 시킨다면, `model.parameters()` 라는 `model.zero_grad()` 는 알아서 모든 parameter들을 업데이트 시켜준다

In [210]:
def fit():
    for epoch in range(epochs):
        for i in range((n-1)//bs+1):
            start_i = i * bs
            endi = start_i +1
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

In [211]:
fit()

- train 후, loss가 내려갔는지 확인 해보자

In [212]:
print(loss_func(model(xb), yb))

tensor(0.0467, grad_fn=<NllLossBackward>)


### nn.Linear 사용하기
### refactor using nn.Linear
- 위에서 부터 계속 우리는 prediction값을 계산할 때 내적 operation을 사용하여 `xb @ weights + bias`를 사용해 왔다. Pytorch의 `nn.Linear`를 사용하여 직접 써 주었던 linear function을 간단하게 바꾸어 보자 

In [213]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10) # input과 output을 parameter로 받는다
        
    def forward(self, xb):
        return self.lin(xb)

In [214]:
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

tensor(2.2596, grad_fn=<NllLossBackward>)


In [215]:
fit()

In [216]:
print(loss_func(model(xb), yb))

tensor(0.0470, grad_fn=<NllLossBackward>)


### torch.optim 사용하기
### refactor using optim
- Pytorch는 `torch.optim`으로 다양한 최적화 알고리즘을 제공한다. 깔끔하게 `step` method를 사용하면 optimizer가 1 step을 나아갈 수 있게 해준다. 그 말은, 
```
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```
이 부분을
```
opt.step()
opt.zero_grad()
``` 
로 한번에 처리를 할 수 있다는 것이다
`optim.zero_grad()` 은 model.zero_grad()와 같이 gradient를 0으로 초기화 시켜주기에 다음 배치를 계산하기 전 실행시켜 주어야 하는 method 이다 

model과 optimizer를 바로 만들어 낼 수 있는 `get_model` 함수를 만들어 보자

In [218]:
from torch import optim

def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr = lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n-1)//bs+1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
print(loss_func(model(xb),yb))

tensor(2.3319, grad_fn=<NllLossBackward>)
tensor(0.0827, grad_fn=<NllLossBackward>)


### Pytorch의 Dataset class를 사용하여 쉽게 각 데이터에 접근하기
### Refactor using dataset
- pytorch의 dataset을 이용하면 각 데이터에 index값으로 쉽게 접근할 수 있다

- 좀 전까지 
``` 
xb = x_train[start_i:end_i]
yb = y_train[start_i:end_i]
```
이 방식으로 데이터와 target 값에 각각 접근하여 training단계에서 mini batch를 가져왔다면,  
`train_ds = TensorDataset(x_train, y_train)`로 train data와 target값을 묶은 후,  
`xb,yb = train_ds[i*bs : i*bs+bs]` 로 한번에 mini batch를 가져올 수 있다


In [5]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(x_train, y_train) 

In [17]:
train_ds[0] # tuple로 data와 target값이 들어가 있다 

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [231]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n-1)//bs+1):
        xb, yb = train_ds[i * bs : i * bs + bs] # DataLoader를 써서 mini batch에 접근
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    print(loss_func(model(xb), yb))

tensor(0.1104, grad_fn=<NllLossBackward>)
tensor(0.0818, grad_fn=<NllLossBackward>)


### DataLoader를 사용하기 
### Refactor using DataLoader
- `DataLoader`는 배치를 다루는 역할을 가지고 있다. 위 처럼 데이터를 통해 만든 `Dataset` 으로 `DataLoader` 를 만들 수 있다.
- `DataLoader`는 iterator 로서 training단계에서 batch를 iterate 할 때 유용하게 쓰인다 

In [232]:
from torch.utils.data import DataLoader
# Dataset으로 만든 train_ds를 DataLoader의 argument로 넣어주고, batch size는 위에서 정한 64를 넣어준다 
train_dl = DataLoader(train_ds, batch_size = bs) 

In [233]:
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl: # dataloader를 iterate할 수 있다 
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()


print(loss_func(model(xb), yb))

tensor(0.0815, grad_fn=<NllLossBackward>)


### Validation set 추가하기 
### Add validation set

- 위에서는 training data로 loss를 확인하는 코드만 작성 했지만, 실제적으로는 **항상** validation set도 함께 loop안에 넣어, validation loss도 체크를 해 주어야 과최적화가 되고있는지에 대한 training상황을 확인할 수 있다
- 맨 위에서 불러온 validation data로 `Dataset`과 `DataLoader`를 만들어 보자

In [234]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size = bs*2)

- 매 epoch마다 validation loss를 계산한 후, 출력해보자 
- validation dataset이 생겼으니, `model.train()`은 training전 항상 켜주어야 하는 스위치 이고, `model.eval()`은 prediction을 하기 전 즉 loss 계산 전, 켜주어야 하는 스위치 임을 기억하자. 이유는, batch normalization `nn.BatchNorm2d` 을 해 주거나, dropout `nn.Dropout` 과 같은 과최적화를 막는 기능들이 training에는 필요한 요소들 이지만, validation에서는 꺼 주어야 하는 요소들이기 때문이다

In [235]:
model, opt = get_model()
for epoch in range(epochs):
    model.train()
    for xb,  yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)
        
    print(epoch, valid_loss / len(valid_dl)) #validation loss 계산 

0 tensor(0.2949)
1 tensor(0.2972)


### `fit()` 과  `get_data()` 만들기
### create `fit()` and `get_data()`
- `fit()` 과  `get_data()` 함수를 만들어 보자! 두개의 함수를 사용하면 위의 training과 validation loss 계산 단계를 3줄로 줄일 수 있다. 
- 먼저 `fit()` 안에서 사용할 `loss_batch` 라는 각 batch의 loss를 계산하는 함수를 만들어 준다. validation set에는 gradient를 계산할 필요가 없으므로, back propagation을 하지 않는다. 그러므로 함수의 parameter `opt`를 parameter로 설정하여, training할때는 켜주고, validation loss를 계산할 때는 꺼준다
- `get_data()`는  Dataset을 받아 DataLoader를 리턴하는 함수이다 

In [242]:
# 각 batch의 loss를 계산
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), len(xb)

In [243]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt) # training할 때는 optimizer 켜주기
        
        model.eval()
        with torch.no_grad(): # evaluation set 에는 gradient 계산 하지 않는다 
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums) # loss의 average계산 
        
        print(epoch, val_loss)

In [244]:
def get_data(train_ds, valid_ds, bs): 
    return (
    DataLoader(train_ds, batch_size=bs, shuffle=True),
    DataLoader(valid_ds, batch_size=bs * 2),
    )

- 단 3줄로 DataLoader 생성에서 부터, 모델을 만들고 training을 하기까지의 과정을 아래와 같이 나타낼 수 있다

In [246]:
epochs = 6
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 0.33802888598442077
1 0.3175259089946747
2 0.3022744032382965
3 0.27204110164642337
4 0.27117016434669494
5 0.27682529377937315


- 다음 Pytorch tutorial에서는 CNN과 같은 특정한 architecture를 실습해 보려 한다